<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import pandas as pd

pickle_file = "features_with_labels.pkl"

with open(pickle_file, "rb") as pkl_file:
  features_with_labels_dic = pickle.load(pkl_file)

print(features_with_labels_dic["joint_angles"][0]["x"]["x_shoulder_to_ellbow_right_axis_angle"])

selected_features = ["COM_to_ground", "knee_angles_right", "knee_angles_left"]
selected_nested_features = ["joint_angles", "axis_angles"]
frames = list(features_with_labels_dic["COM_to_ground"].keys())
labels = [features_with_labels_dic["schwung_labels"][frame] for frame in frames]

print(frames)

data = {"Frame": frames, "Label": labels}

for feature in selected_features:
  data[feature] = [features_with_labels_dic[feature][frame] for frame in frames]

for feature in selected_nested_features:
  for frame in frames:
    for coordinate in features_with_labels_dic[feature][frame]:
      for feature_name in features_with_labels_dic[feature][frame][coordinate]:
        if feature_name not in data:
          data[feature_name] = []  # Falls nicht vorhanden, erstelle eine Liste
        data[feature_name].append(features_with_labels_dic[feature][frame][coordinate][feature_name])

print(list(data.keys()))

pd.set_option("display.max_columns", None)
df = pd.DataFrame(data)
df.head()




FileNotFoundError: [Errno 2] No such file or directory: 'features_with_labels.pkl'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

selected_features = [col for col in df.columns if col not in ["Frame", "Label"]]

X = df[selected_features].values.reshape(1, -1, len(selected_features))
Y = df["Label"].values.reshape(1, -1)

X_train = torch.tensor(X, dtype=torch.float32)
Y_train = torch.tensor(Y, dtype=torch.long)

class MyLSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size):
     super(MyLSTMModel, self).__init__()
     self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
     self.fc = nn.Linear(hidden_size, output_size)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 🔹 **Alle Features außer Frame & Label als Input**
selected_features = [col for col in df.columns if col not in ["Frame", "Label"]]

# **LSTM Input vorbereiten** -> reshape ist aufgebaut wie folgt (Anzahl Videos, Anzahl Frames (-1 berechnet Python Anzahl automatisch), Anzahl Features)
X = df[selected_features].values.reshape(1, -1, len(selected_features))  # (Batch=1, Time, Features)
Y = df["Label"].values.reshape(1, -1)  # (Batch=1, Time)

# **In Torch-Tensor umwandeln**
X_train = torch.tensor(X, dtype=torch.float32)
Y_train = torch.tensor(Y, dtype=torch.long)

# 🔹 **LSTM-Modell definieren**
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)
        return out

# 🔹 **Modell initialisieren**
input_size = len(selected_features)  # **Alle Features als Input**
hidden_size = 64
num_layers = 2
output_size = 3  # **0 = Kein Schwung, 1 = Linksschwung, 2 = Rechtsschwung**

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 🔹 **Loss & Optimizer**
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🔹 **Training**
num_epochs = 50
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(0), Y_train.squeeze(0))
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training abgeschlossen! 🚀")


Epoch 0, Loss: 1.2369199991226196
Epoch 10, Loss: 0.9103463292121887
Epoch 20, Loss: 0.6348735690116882
Epoch 30, Loss: 0.3476511836051941
Epoch 40, Loss: 0.14514248073101044
Training abgeschlossen! 🚀


In [ ]:
with torch.no_grad():  # Keine Gradientenberechnung nötig
    predictions = model(X_train)
    predicted_classes = torch.argmax(predictions, dim=2)  # Klasse mit höchster Wahrscheinlichkeit
print(predicted_classes)


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
correct = (predicted_classes.squeeze(0) == Y_train.squeeze(0)).sum().item()
accuracy = correct / len(Y_train.squeeze(0))
print(f"Modellgenauigkeit: {accuracy:.2f}")


Modellgenauigkeit: 0.98


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 🔹 **Alle Features außer Frame & Label als Input**
selected_features = [col for col in df.columns if col not in ["Frame", "Label"]]

# **GRU Input vorbereiten**
X = df[selected_features].values.reshape(1, -1, len(selected_features))  # (Batch=1, Time, Features)
Y = df["Label"].values.reshape(1, -1)  # (Batch=1, Time)

# **In Torch-Tensor umwandeln**
X_train = torch.tensor(X, dtype=torch.float32)
Y_train = torch.tensor(Y, dtype=torch.long)

# 🔹 **GRU-Modell definieren**
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        out = self.fc(gru_out)
        return out

# 🔹 **Modell initialisieren**
input_size = len(selected_features)  # **Alle Features als Input**
hidden_size = 64
num_layers = 2
output_size = 3  # **0 = Kein Schwung, 1 = Linksschwung, 2 = Rechtsschwung**

model = GRUModel(input_size, hidden_size, num_layers, output_size)

# 🔹 **Loss & Optimizer**
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🔹 **Training**
num_epochs = 50
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(0), Y_train.squeeze(0))
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training abgeschlossen! 🚀")


Epoch 0, Loss: 1.0510342121124268
Epoch 10, Loss: 0.5786163210868835
Epoch 20, Loss: 0.42098215222358704
Epoch 30, Loss: 0.22384342551231384
Epoch 40, Loss: 0.08048948645591736
Training abgeschlossen! 🚀


In [ ]:
with torch.no_grad():  # Keine Gradientenberechnung nötig
    predictions_gru = model(X_train)  # Nutze "model" statt "gru_model"
    predicted_classes_gru = torch.argmax(predictions_gru, dim=2)  # Klasse mit höchster Wahrscheinlichkeit
print(predicted_classes_gru)


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
with torch.no_grad():
    predictions_gru = model(X_train)
    predicted_classes_gru = torch.argmax(predictions_gru, dim=2)

correct_gru = (predicted_classes_gru.squeeze(0) == Y_train.squeeze(0)).sum().item()
accuracy_gru = correct_gru / len(Y_train.squeeze(0))
print(f"GRU-Modellgenauigkeit: {accuracy_gru:.2f}")


GRU-Modellgenauigkeit: 0.99
